In [5]:
import time
import random
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


# Medicine Class
class Medicine:
    def __init__(self, name, batch_no, mfg_date, exp_date, chemical_composition, temperature, humidity):
        self.name = name
        self.batch_no = batch_no
        self.mfg_date = mfg_date
        self.exp_date = exp_date
        self.chemical_composition = chemical_composition
        self.temperature = temperature
        self.humidity = humidity
        self.quality_status = "Unknown"

    def check_quality(self):
        if self.temperature > 30 or self.humidity > 70:
            self.quality_status = "Poor"
        elif self.temperature < 15 or self.humidity < 20:
            self.quality_status = "Critical"
        else:
            self.quality_status = "Good"
        return self.quality_status


# Monitor Quality
def monitor_quality(products):
    for product in products:
        status = product.check_quality()
        print(f"Product: {product.name}, Batch: {product.batch_no}, Status: {status}")


# Alert System
def alert_system(product):
    if product.quality_status != "Good":
        print(f"ALERT: {product.name} (Batch {product.batch_no}) is in {product.quality_status} condition!")


# Monitoring Cycle
def run_monitoring_cycle(products, cycles=3):
    for i in range(cycles):
        print(f"\n--- Monitoring Cycle {i+1} ---")
        for product in products:
            product.temperature = random.uniform(10, 40)  # Randomize temperature
            product.humidity = random.uniform(10, 80)  # Randomize humidity
            product.check_quality()
            alert_system(product)
        time.sleep(2)


# Sample Medicines
med1 = Medicine("Paracetamol", "B123", "01/01/2024", "01/01/2026", "C8H9NO2", 25, 50)
med2 = Medicine("Ibuprofen", "B456", "15/03/2024", "15/03/2026", "C13H18O2", 20, 48)

products = [med1, med2]
run_monitoring_cycle(products, cycles=3)

# Training ML Model for Quality Prediction
X_train = np.array([[25, 40], [32, 75], [20, 50], [35, 80], [15, 30]])
y_train = np.array([1, 0, 1, 0, 1])  # (1: Good, 0: Bad)

model = LogisticRegression()
model.fit(X_train, y_train)


def predict_quality(temperature, humidity):
    prediction = model.predict([[temperature, humidity]])[0]
    return "Good" if prediction == 1 else "Bad"


predicted_status = predict_quality(22, 51)
print("Predicted Quality Status:", predicted_status)


# Create Database
def create_database():
    conn = sqlite3.connect("drug_inventory.db")
    cursor = conn.cursor()

    cursor.executescript("""
        CREATE TABLE IF NOT EXISTS drugs (
            drug_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            batch_number TEXT UNIQUE NOT NULL,
            supplier_id INTEGER,
            quantity INTEGER NOT NULL CHECK(quantity >= 0),
            expiry_date DATE NOT NULL,
            location TEXT NOT NULL,
            status TEXT CHECK(status IN ('In Stock', 'Out of Stock', 'Expired')),
            FOREIGN KEY (supplier_id) REFERENCES suppliers(supplier_id)
        );

        CREATE TABLE IF NOT EXISTS suppliers (
            supplier_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            contact_name TEXT NOT NULL,
            phone TEXT NOT NULL,
            email TEXT NOT NULL UNIQUE,
            address TEXT NOT NULL
        );

        CREATE TABLE IF NOT EXISTS supply_chain (
            transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
            drug_id INTEGER NOT NULL,
            supplier_id INTEGER NOT NULL,
            transaction_type TEXT CHECK(transaction_type IN ('Received', 'Distributed')),
            quantity INTEGER NOT NULL CHECK(quantity > 0),
            transaction_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            destination TEXT,
            FOREIGN KEY (drug_id) REFERENCES drugs(drug_id),
            FOREIGN KEY (supplier_id) REFERENCES suppliers(supplier_id)
        );

        CREATE TABLE IF NOT EXISTS stock_logs (
            log_id INTEGER PRIMARY KEY AUTOINCREMENT,
            drug_id INTEGER NOT NULL,
            change_type TEXT CHECK(change_type IN ('Added', 'Removed', 'Updated')),
            previous_quantity INTEGER NOT NULL,
            new_quantity INTEGER NOT NULL,
            change_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (drug_id) REFERENCES drugs(drug_id)
        );
    """)

    conn.commit()
    conn.close()
    print("Database and tables created successfully!")


create_database()


# Fetch Supply Chain Data
def get_supply_chain_data():
    conn = sqlite3.connect('drug_inventory.db')
    cursor = conn.cursor()
    cursor.execute('''
        SELECT d.name, s.transaction_date, s.quantity 
        FROM supply_chain s 
        JOIN drugs d ON s.drug_id = d.drug_id
    ''')
    rows = cursor.fetchall()
    conn.close()
    return rows


# Plot Supply Chain Data
def plot_supply_chain_movements():
    data = get_supply_chain_data()
    if not data:
        print("No data available for plotting.")
        return

    drug_names = [row[0] for row in data]
    dates = [datetime.strptime(row[1], "%Y-%m-%d %H:%M:%S") for row in data]
    quantities_moved = [row[2] for row in data]

    plt.figure(figsize=(10, 6))
    plt.plot(dates, quantities_moved, marker='o', linestyle='-', color='orange')
    plt.title('Supply Chain Movement Over Time')
    plt.xlabel('Date')
    plt.ylabel('Quantity Moved')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


plot_supply_chain_movements()




--- Monitoring Cycle 1 ---
ALERT: Ibuprofen (Batch B456) is in Poor condition!

--- Monitoring Cycle 2 ---
ALERT: Paracetamol (Batch B123) is in Poor condition!
ALERT: Ibuprofen (Batch B456) is in Poor condition!

--- Monitoring Cycle 3 ---
Predicted Quality Status: Good
Database and tables created successfully!
No data available for plotting.
